<a href="https://colab.research.google.com/github/nalika/NLP/blob/master/keras_lstm_word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-6).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [3]:
# Mount gdrive
#---------------
from google.colab import drive 
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
!tar -xvf ('/content/gdrive/My Drive/Colab Notebooks/samples-examples.tgz')

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `tar -xvf ('/content/gdrive/My Drive/Colab Notebooks/samples-examples.tgz')'


In [0]:
data_path = '/root/data/ptb/simple-examples/data/'

In [8]:
# Get the Rossmann data from the fast.ai URL, and make a nested directory to put it in later. 
# -p flag from mkdir is to make a parent directory (allows nested directories to be created at once)
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz && mkdir -p ~/data/ptb

--2019-07-31 00:11:34--  http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
Resolving www.fit.vutbr.cz (www.fit.vutbr.cz)... 147.229.9.23, 2001:67c:1220:809::93e5:917
Connecting to www.fit.vutbr.cz (www.fit.vutbr.cz)|147.229.9.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34869662 (33M) [application/x-gtar]
Saving to: ‘simple-examples.tgz’

simple-examples.tgz 100%[===================>]  33.25M  1.04MB/s    in 55s     

2019-07-31 00:12:30 (621 KB/s) - ‘simple-examples.tgz’ saved [34869662/34869662]



In [0]:
# Unzip the .tgz file
# x for extract
# -v for verbose    # NOTE: I usually turn this off; it prints a lot...
# -z for gnuzip
# -f for file (should come at last just before file name)
# -C to extract the zipped contents to a different directory
!tar -xzf simple-examples.tgz -C ~/data/ptb/

In [0]:
# Remove the .tgz file
!rm simple-examples.tgz

In [11]:
# Make sure the data's where we think it is:
!ls -ltr /root/data/ptb/simple-examples/

total 52
drwx------ 3 491 root 4096 Apr 29  2011 2-nbest-rescore
drwx------ 2 491 root 4096 May  1  2011 3-combination
drwx------ 2 491 root 4096 May  1  2011 4-data-generation
drwx------ 2 491 root 4096 May  1  2011 1-train
drwx------ 2 491 root 4096 May  1  2011 5-one-iter
drwx------ 2 491 root 4096 May  1  2011 6-recovery-during-training
drwx------ 2 491 root 4096 May  1  2011 7-dynamic-evaluation
drwx------ 2 491 root 4096 May  1  2011 8-direct
drwx------ 2 491 root 4096 May  2  2011 9-char-based-lm
drwx------ 2 491 root 4096 May  2  2011 temp
drwx------ 2 491 root 4096 May  2  2011 models
drwx------ 2 491 root 4096 May  2  2011 rnnlm-0.2b
drwx------ 2 491 root 4096 Sep 13  2011 data


In [12]:
!ls

gdrive	sample_data


The load_data function which I created to run these functions is shown below:

In [13]:
import collections
import os
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed
from keras.layers import LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import argparse

Using TensorFlow backend.


In [0]:


def read_words(filename):
    with tf.gfile.GFile(filename, "r") as f:
        #return f.read().decode("utf-8").replace("\n", "<eos>").split()
         return f.read().replace("\n", "<eos>").split()


def build_vocab(filename):
    data = read_words(filename)

    counter = collections.Counter(data)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))

    return word_to_id


def file_to_word_ids(filename, word_to_id):
    data = read_words(filename)
    return [word_to_id[word] for word in data if word in word_to_id]

In [0]:
def load_data():
    # get the data paths
    train_path = os.path.join("/root/data/ptb/simple-examples/data/", "ptb.train.txt")
    valid_path = os.path.join("/root/data/ptb/simple-examples/data/", "ptb.valid.txt")
    test_path = os.path.join("/root/data/ptb/simple-examples/data/", "ptb.test.txt")

    # build the complete vocabulary, then convert text data to list of integers
    word_to_id = build_vocab(train_path)
    train_data = file_to_word_ids(train_path, word_to_id)
    valid_data = file_to_word_ids(valid_path, word_to_id)
    test_data = file_to_word_ids(test_path, word_to_id)
    vocabulary = len(word_to_id)
    reversed_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))

    print(train_data[:5])
    print(word_to_id)
    print(vocabulary)
    print(" ".join([reversed_dictionary[x] for x in train_data[:10]]))
    return train_data, valid_data, test_data, vocabulary, reversed_dictionary

Call this function

In [17]:
train_data, valid_data, test_data, vocabulary, reversed_dictionary = load_data()


[9970, 9971, 9972, 9974, 9975]
{'the': 0, '<unk>': 1, '<eos>': 2, 'N': 3, 'of': 4, 'to': 5, 'a': 6, 'in': 7, 'and': 8, "'s": 9, 'that': 10, 'for': 11, '$': 12, 'is': 13, 'it': 14, 'said': 15, 'on': 16, 'by': 17, 'at': 18, 'as': 19, 'from': 20, 'million': 21, 'with': 22, 'mr.': 23, 'was': 24, 'be': 25, 'are': 26, 'its': 27, 'he': 28, 'but': 29, 'has': 30, 'an': 31, "n't": 32, 'will': 33, 'have': 34, 'new': 35, 'or': 36, 'company': 37, 'they': 38, 'this': 39, 'year': 40, 'which': 41, 'would': 42, 'about': 43, 'says': 44, 'more': 45, 'were': 46, 'market': 47, 'billion': 48, 'his': 49, 'had': 50, 'their': 51, 'up': 52, 'u.s.': 53, 'one': 54, 'than': 55, 'who': 56, 'some': 57, 'been': 58, 'also': 59, 'stock': 60, 'other': 61, 'share': 62, 'not': 63, 'we': 64, 'corp.': 65, 'if': 66, 'when': 67, 'i': 68, 'last': 69, 'president': 70, 'shares': 71, 'years': 72, 'all': 73, 'first': 74, 'two': 75, 'because': 76, 'trading': 77, 'after': 78, 'could': 79, 'inc.': 80, 'sales': 81, '&': 82, 'there': 8

**NOTE:** As you can observe, the training data is comprised of a list of integers, as expected.

the output vocabulary is simply the size of our text corpus. When words are incorporated into the training data, every single unique word is not considered – rather, in natural language processing, the text data is usually limited to a certain N number of the most common words. In this case N = vocabulary = 10,000.

Keras has  some handy functions which can extract training data automatically from a pre-supplied Python iterator/generator object and input it to the model. One of these Keras functions is called fit_generator. 

The Python iterator function needs to have a form like:

In [0]:
class KerasBatchGenerator(object):

    def __init__(self, data, num_steps, batch_size, vocabulary, skip_step=5):
        self.data = data
        self.num_steps = num_steps
        self.batch_size = batch_size
        self.vocabulary = vocabulary
        # this will track the progress of the batches sequentially through the
        # data set - once the data reaches the end of the data set it will reset
        # back to zero
        self.current_idx = 0
        # skip_step is the number of words which will be skipped before the next
        # batch is skimmed from the data set
        self.skip_step = skip_step

    def generate(self):
        x = np.zeros((self.batch_size, self.num_steps))
        y = np.zeros((self.batch_size, self.num_steps, self.vocabulary))
        while True:
            for i in range(self.batch_size):
                if self.current_idx + self.num_steps >= len(self.data):
                    # reset the index back to the start of the data set
                    self.current_idx = 0
                x[i, :] = self.data[self.current_idx:self.current_idx + self.num_steps]
                temp_y = self.data[self.current_idx + 1:self.current_idx + self.num_steps + 1]
                # convert all of temp_y into a one hot representation
                y[i, :, :] = to_categorical(temp_y, num_classes=self.vocabulary)
                self.current_idx += self.skip_step
            yield x, y


### Creating the Keras LSTM structure

In [20]:
num_steps = 30
hidden_size = 500
use_dropout=True

model = Sequential()
model.add(Embedding(vocabulary, hidden_size, input_length=num_steps))
model.add(LSTM(hidden_size, return_sequences=True))
model.add(LSTM(hidden_size, return_sequences=True))
if use_dropout:
    model.add(Dropout(0.5))
model.add(TimeDistributed(Dense(vocabulary)))
model.add(Activation('softmax'))

W0731 00:16:15.039422 140495789832064 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0731 00:16:15.052385 140495789832064 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0731 00:16:16.654175 140495789832064 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0731 00:16:16.667336 140495789832064 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Ins

The first step involves creating a Keras model with the Sequential() constructor, is a word embedding layer. 

This will convert our words (referenced by integers in the data) into meaningful embedding vectors. It takes the size of the vocabulary as its first argument, then the size of the resultant embedding vector that you want as the next argument. 

Finally, because this layer is the first layer in the network, we must specify the “length” of the input i.e. the number of steps/words in each sample

In [0]:
num_steps = 30
batch_size = 20

train_data_generator = KerasBatchGenerator(train_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)
valid_data_generator = KerasBatchGenerator(valid_data, num_steps, batch_size, vocabulary,
                                           skip_step=num_steps)

Compile the model

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


W0731 00:16:29.672808 140495789832064 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0731 00:16:29.708853 140495789832064 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



Call backs and save path

In [0]:
mkdir -p ~/data/keras_lstm_w2v

In [28]:
!ls /root/data/

keras_lstm_w2v	ptb


In [0]:
checkpointer = ModelCheckpoint(filepath="/root/data/keras_lstm_w2v" + '/model-{epoch:02d}.hdf5', verbose=1)


### Training the Keras LSTM model

**NOTE:**  PTB data set is a serious text data set – not a toy problem to demonstrate how good LSTM models are. Therefore, in order to get good results, you’ll likely have to run the model over many epochs, and the model will need to have a significant level of complexity.

Therefore, it is likely to take a long time on a CPU machine, and I’d suggest running it on a machine with a **good GPU** if you want to try and replicate things.

In [30]:
num_epochs = 10

model.fit_generator(train_data_generator.generate(), len(train_data)//(batch_size*num_steps), num_epochs,
                        validation_data=valid_data_generator.generate(),
                        validation_steps=len(valid_data)//(batch_size*num_steps), callbacks=[checkpointer])

Epoch 1/10
1549/1549 [==============================] - 338s 218ms/step - loss: 6.3311 - categorical_accuracy: 0.1049 - val_loss: 6.0533 - val_categorical_accuracy: 0.1451

Epoch 00001: saving model to /root/data/keras_lstm_w2v/model-01.hdf5
Epoch 2/10
1549/1549 [==============================] - 337s 218ms/step - loss: 6.2056 - categorical_accuracy: 0.1178 - val_loss: 5.9102 - val_categorical_accuracy: 0.1590

Epoch 00002: saving model to /root/data/keras_lstm_w2v/model-02.hdf5
Epoch 3/10
1549/1549 [==============================] - 337s 217ms/step - loss: 6.1067 - categorical_accuracy: 0.1264 - val_loss: 5.8541 - val_categorical_accuracy: 0.1642

Epoch 00003: saving model to /root/data/keras_lstm_w2v/model-03.hdf5
Epoch 4/10
1549/1549 [==============================] - 338s 218ms/step - loss: 6.0527 - categorical_accuracy: 0.1295 - val_loss: 5.8416 - val_categorical_accuracy: 0.1670

Epoch 00004: saving model to /root/data/keras_lstm_w2v/model-04.hdf5
Epoch 5/10
1549/1549 [==========

### Testing

In [36]:
# test data set
    dummy_iters = 40
    example_test_generator = KerasBatchGenerator(test_data, num_steps, 1, vocabulary,
                                                     skip_step=1)
    print("Test data:")
    for i in range(dummy_iters):
        dummy = next(example_test_generator.generate())
    num_predict = 10
    true_print_out = "Actual words: "
    pred_print_out = "Predicted words: "
    for i in range(num_predict):
        data = next(example_test_generator.generate())
        #print(data[0].shape)
        prediction = model.predict(data[0])
        predict_word = np.argmax(prediction[:, num_steps - 1, :])
        true_print_out += reversed_dictionary[test_data[num_steps + dummy_iters + i]] + " "
        pred_print_out += reversed_dictionary[predict_word] + " "
    print(true_print_out)
    print(pred_print_out)

Test data:
Actual words: futures <eos> the N stock specialist firms on the big 
Predicted words: the <eos> the <unk> N market said <eos> the <unk> 
